In [213]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

In [214]:
train_data = datasets.FashionMNIST(root="data",
                                   train = True,
                                   download=False,
                                   transform=ToTensor(),
                                   target_transform=None)

test_data = datasets.FashionMNIST(root="data",
                                  download=False,
                                  train=False,
                                  transform=ToTensor(),
                                  target_transform=None)

In [215]:
len(train_data),len(test_data)

(60000, 10000)

In [216]:
batch_size=32


train_data_loader = torch.utils.data.DataLoader(batch_size=batch_size,
                                          shuffle=True,
                                          dataset=train_data)

test_data_loader = torch.utils.data.DataLoader(dataset=test_data,
                                               shuffle=False,
                                               batch_size=batch_size)

In [217]:
len(train_data_loader),len(test_data_loader)

(1875, 313)

In [218]:
classes = train_data.classes
classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [219]:
len(train_data_loader), len(test_data_loader)


(1875, 313)

In [220]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [221]:
class CNN_mode(nn.Module):
    def __init__(self, input_shape:int,
                 output_shape : int,
                 hidden_units : int):
        super().__init__()
        self.conv_black1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_black2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,
                                    out_features=output_shape)

        )
    
    def forward(self,x):
        x = self.conv_black1(x)
        x = self.conv_black2(x)
        x = self.classifier(x)
        return x

In [222]:
torch.manual_seed(42)

model = CNN_mode(input_shape=1, output_shape=len(classes), hidden_units=10)

model.to(device)

CNN_mode(
  (conv_black1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_black2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [223]:
from helper_functions import accuracy_fn


loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)

In [224]:
def train_model(model:nn.Module,
                data_loader:torch.utils.data.DataLoader,
                optimizer:torch.optim.Optimizer,
                loss_fn:torch.nn.Module,
                accuracy_fn,
                device:torch.device=device):
    train_loss, train_acc = 0, 0

    model.train()

    for batch, (x, y) in enumerate(data_loader):
        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)
        loss = loss_fn(y_pred, y)

        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"train loss : {train_loss:.5f} | train acc : {train_acc:.5f}%")


In [225]:
def test_model(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:nn.Module,
               accuracy_fn,
               device:torch.device):
    
    test_loss, test_acc = 0, 0

    model.eval()
    with torch.inference_mode():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)
            
            test_pred = model(x)

           
        

            test_loss +=  loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"test loss : {test_loss:.5f}  | test acc : {test_acc:.5f}%")



In [226]:
from timeit import default_timer as timer

def print_train_time(start:float, end:float, device:torch.device=None):

  total_time = end - start
  print(f"Train time of {device} : {total_time:.3f} seconds...")
  return total_time

In [227]:
from tqdm.auto import tqdm
from timeit import default_timer as timer


torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 3
train_time_start_model = timer()

for epoch in tqdm(range(epochs)):
    print(f"epochs : {epoch} \n_____")

    train_model(model=model,
                data_loader=train_data_loader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                accuracy_fn=accuracy_fn,
                device=device)
    test_model(model=model,
               data_loader=test_data_loader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_fn,
               device=device)
    
train_time_end_model = timer()

total_train_time_model = print_train_time(end=train_time_end_model,start=train_time_start_model,device=device)


  0%|          | 0/3 [00:00<?, ?it/s]

epochs : 0 
_____
train loss : 0.61922 | train acc : 77.54167%
test loss : 0.40464  | test acc : 85.74281%
epochs : 1 
_____
train loss : 0.36875 | train acc : 86.79167%
test loss : 0.35132  | test acc : 87.45008%
epochs : 2 
_____
train loss : 0.33087 | train acc : 87.97167%
test loss : 0.33205  | test acc : 87.75958%
Train time of cuda : 21.532 seconds...
